#### Importing necessary libraries

In [14]:
import numpy as np
import pandas as pd 
import os
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

In [2]:
order_final = pd.read_csv('order_final.csv')

### Applying Dimensinality Reduction on User Ordered Products

In [3]:
order_final.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,...,product_ordered_count,product_reordered_count,product_reorder_ratio,time_btwn_next_order,department_ordered_count,department_reordered_count,department_reorder_ratio,aisle_ordered_count,aisle_reordered_count,aisle_reorder_ratio
0,0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,...,8,7,0.875000,9.0,217051,146502,0.674966,55240,37939,0.686803
1,1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,...,144,107,0.743056,9.0,217051,146502,0.674966,12820,7493,0.584477
2,2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,...,1062,569,0.535782,9.0,409087,271886,0.664617,150609,91368,0.606657
3,3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,...,2413,1709,0.708247,9.0,409087,271886,0.664617,150609,91368,0.606657
4,4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,...,24,10,0.416667,9.0,46799,22782,0.486805,3241,1644,0.507251


In [4]:
# Reducing the data due to continous memory errors
order_final = order_final[:250000]

In [5]:
order_final.shape

(250000, 27)

In [6]:
user_ordered_product = pd.crosstab(order_final['user_id'],order_final['product_name'])
user_ordered_product.head()

product_name,#2 Coffee Filters,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,0% Fat Superfruits Greek Yogurt,...,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
user_id,,,,,,,,,,,,,,,,,,,,,
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
user_ordered_product.shape

(23648, 24256)

In [9]:
user_ordered_product.describe()

product_name,#2 Coffee Filters,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,0% Fat Superfruits Greek Yogurt,...,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
count,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,...,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000,23648.000000
mean,0.000169,0.000042,0.000042,0.000085,0.000085,0.000211,0.001353,0.000888,0.000127,0.000211,...,0.001311,0.000169,0.000127,0.000127,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042
std,0.013005,0.006503,0.006503,0.009196,0.009196,0.014540,0.036761,0.029787,0.011263,0.014540,...,0.036183,0.013005,0.011263,0.011263,0.006503,0.006503,0.006503,0.006503,0.006503,0.006503
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
X = user_ordered_product.values

### Finding the best dimension using elbow technique

In [ ]:

distortions = []
K = range(1, 20)
 
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X) 
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,'euclidean'), axis=1)) / X.shape[0])
    

In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('K value')
plt.ylabel('Distortion')
plt.title('Elbow Method using Distortion')
plt.show()